<a href="https://colab.research.google.com/github/Frienddddddddd/mask/blob/main/Image_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install -q kaggle
import json

username = "suchanaree"
key = "5ba00f6bc2b2c01aae5726d1722751b3"
data = {"username": username, "key": key}
json_data = json.dumps(data)

# Write the JSON string to a file
file_path = "/content/kaggle.json"
with open(file_path, 'w') as json_file:
    json_file.write(json_data)

In [10]:
!mkdir ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/ #copy api key ---- depend on your directory -- my directory is .../colab/..
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list
!kaggle competitions download -c image-search #download competition dataset

mkdir: cannot create directory ‘/root/.kaggle’: File exists
ref                                                                title                                               size  lastUpdated          downloadCount  voteCount  usabilityRating  
-----------------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
nelgiriyewithana/apple-quality                                     Apple Quality                                      170KB  2024-01-11 14:31:07           7658        185  1.0              
hummaamqaasim/jobs-in-data                                         Jobs and Salaries in Data Science                   76KB  2023-12-25 20:03:32          13787        250  1.0              
willianoliveiragibin/qs-top-100-universities                       QS top 100 universities                              4KB  2024-01-21 20:40:02            908         22  1.0     

In [ ]:
!unzip /content/image-search.zip

Archive:  /content/image-search.zip
replace queries/queries/0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
from transformers import (
    AutoModel,
    CLIPProcessor
)
import os
import pandas as pd
from PIL import Image
import torch


if __name__ == '__main__':
    src_dir = 'test/images'
    query_dir = 'queries/queries'
    submission = pd.read_csv('sample_submission.csv')
    model = AutoModel.from_pretrained('openai/clip-vit-large-patch14-336').cuda().eval()
    processor = CLIPProcessor.from_pretrained('openai/clip-vit-large-patch14-336')

    submission['dot_class'] = 22
    submission['cosine_class'] = 22
    with torch.no_grad():
        query_images = []
        query_classes = []
        for file in os.listdir(query_dir):
            inputs = processor(images=[Image.open(os.path.join(query_dir, file)).convert('RGB')], return_tensors='pt').to('cuda')
            outputs = model.get_image_features(inputs.pixel_values).cpu()
            outputs = outputs / outputs.norm(p=2, dim=-1, keepdim=True)
            query_images.append(outputs)
            query_classes.append(int(file[:-4]))
        query_images = torch.cat(query_images)
        for idx, row in submission.iterrows():
            if not pd.isna(row['class']):
                continue
            inputs = processor(images=[Image.open(os.path.join(src_dir, row['img_file'])).convert('RGB')], return_tensors='pt').to('cuda')
            outputs = model.get_image_features(inputs.pixel_values).cpu()
            outputs = outputs / outputs.norm(p=2, dim=-1, keepdim=True)
            values = outputs @ query_images.T
            if values.softmax(1).max() > .055:
                submission.at[idx, 'dot_class'] = query_classes[values.argmax().numpy().tolist()]
            cosine = torch.cosine_similarity(outputs, query_images)
            if cosine.max() > 0.8:
                submission.at[idx, 'cosine_class'] = query_classes[cosine.argmax().numpy().tolist()]

        sub = submission[['img_file',]]
        sub['class'] = submission['dot_class']
        sub.to_csv('dot_product.csv', index=False)
        sub['class'] = submission['cosine_class']
        sub.to_csv('cosine_similarity.csv', index=False)


In [ ]:
!kaggle competitions submit -c image-search -f /content/dot_product.csv -m "dot product openai/clip-vit-large-patch14-336"